## 简答题
1. 如果训练集有100万个实例，训练决策树（无约束）大致的深度是多少？

log_2 (100_0000)

2. 通常来说，子节点的基尼杂质是高于还是低于其父节点？是通常更高/更低？还是永远更高/更低？

通常来说子节点的基尼杂志低于父节点，是通常更低。因此在一些情况下分裂对纯度无改善，子节点的基尼杂志可能与父节点相同

3. 如果决策树过拟合训练集，减少max_depth是否为一个好主意？

是一个好主意，减小最大深度会减少节点的分裂，从而减低拟合程度，这是一种常用的正则化方法。

4. 如果决策树对训练集欠拟合，尝试缩放输入特征是否为一个好主意？

不是，决策树是基于特征阈值分裂的模型，对特征缩放不敏感。欠拟合可以通过增加max_depth，减少min_samples_split来增加模型的拟合程度

5. 如果在给定的训练集上训练决策树需要一个小时，那么如果将特征数量变为两倍，训练大约需要多少时间？

决策数的复杂度为 树深度×log2(实例数)×特征数×实例数

因此，将特征树翻倍，训练时间也就会翻倍

6. 如果在包含100万个实例的训练集上训练决策树需要一个小时，那么在包含1000万个实例的训练集上训练决策树，大概需要多长时间？提示：考虑CART算法的计算复杂度。

树深度×log2(实例数)×特征数×实例数 = 1 h

树深度×log2(实例数*10)×特征数×实例数*10 ≈ 11.67 h





## 编程题

1. 为 新月形 数据集训练并微调一棵决策树。

a. 使用make_moons(n_samples=10000，noise=0.4)生成一个 新月形 数据集。

b. 使用train_test_split()拆分训练集和测试集。

c. 使用交叉验证的网格搜索（在GridSearchCV类的帮助下）为Decision-TreeClassifier找到适合的超参数值。提示：尝试max_leaf_nodes的多种值。

d. 使用超参数对整个训练集进行训练，并测量模型在测试集上的性能。你应该得到约85%～87%的准确率。

In [72]:
from nltk import accuracy
from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier

X,y = make_moons(n_samples=10000,noise = 0.4)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
grid_parm = {
    "max_depth" : [3,5,7],
    "min_samples_split" : [2,6,12],
    "min_samples_leaf" : [3,6,9],
    "max_leaf_nodes" : [2,4,6,8,10],
}
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid=grid_parm,cv = 5,scoring="accuracy",n_jobs=-1)
grid_search.fit(X_train ,y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': [3, 5, 7],
                         'max_leaf_nodes': [2, 4, 6, 8, 10],
                         'min_samples_leaf': [3, 6, 9],
                         'min_samples_split': [2, 6, 12]},
             scoring='accuracy')

In [73]:
best_model  =  grid_search.best_estimator_
print(best_model.score(X_train,y_train))
grid_search.best_params_

0.867125


{'max_depth': 3,
 'max_leaf_nodes': 4,
 'min_samples_leaf': 3,
 'min_samples_split': 2}

In [74]:
dtc = DecisionTreeClassifier(max_depth= 3,max_leaf_nodes= 4,min_samples_split= 2,min_samples_leaf= 3).fit(X_train,y_train)
dtc.score(X_train,y_train)

0.867125

In [75]:
from sklearn.metrics import accuracy_score

one = dtc.predict(X_test)
accuracy_score(y_test,one)

0.8465

2. 按照以下步骤种植森林。

a.继续之前的练习，生产1000个训练集子集，每个子集包含随机挑选的100个实例。提示：使用Scikit-Learn的ShuffleSplit类来实现。

b.使用前面得到的最佳超参数值，在每个子集上训练一棵决策树。在测试集上评估这1000棵决策树。因为训练集更小，所以这些决策树的表现可能比第一棵决策树要差一些，只能达到约80%的精度。

c.见证奇迹的时刻到了。用每个测试集实例，生成1000棵决策树的预测，然后仅保留次数最频繁的预测［可以使用SciPy的mode()函数］。这样你在测试集上可获得大多数投票的预测结果。

d.评估测试集上的这些预测，你得到的准确率应该比第一个模型更高（高出0.5%～1.5%）。恭喜，你已经训练出了一个随机森林分类器！

In [76]:
import numpy as np
from scipy.stats import mode
from sklearn.model_selection import  ShuffleSplit

# 先划分固定测试集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

predictions  = []
sfsp = ShuffleSplit(n_splits=1000, train_size=100, random_state=42)

for train_index, _ in sfsp.split(X_train):
    dtc = DecisionTreeClassifier(max_depth= 3,max_leaf_nodes= 4,min_samples_split= 2,min_samples_leaf= 3)
    dtc.fit(X_train[train_index], y_train[train_index])
    predictions.append(dtc.predict(X_test))
final_pred = mode(np.array(predictions ), axis=0)[0]

In [77]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,final_pred)

0.867